
## HAP Delta One Data Analysis Exercise
***
### __Candidate Instructions__
You will be given two data sets with Olympic Medal data and Country GDP data and asked to first do some analytics and then some predictions based on patterns in the data. Please make sure to have Pandas in your Python environment in order to access the data.

In the cell below, you will see a simple loading of two datasets, historical Olympic medal data and Country GDP per capita data.
+ Olympic Medal Data
  + __Season__: Designating the two types of olympics Winter and Summer games. These are more recently held every four years as the data indicates.
  + __NOC__: 3 Letter Country Code Designated by the Olympic Committee
+ GDP per Capita (Country) data
  + __Country Code__: 3 Digit Code identifying the country (Also known as ISO 3166-1 alpha-3)
  
For the below instructions, parsing and analysis must be done in Python. The Presentation of analysis can be done on any platform (e.g. iPython/Jupyter notebook, Word, Excel).

All work submitted will be considered; how you arrive at the results is as important as the results themselves.

Only entries sent within two hours of the time this email was sent will be considered.

### Part 1: Data Description
+ What do you notice about the quality of the data? Why do you think this is the case?
+ Which country has won the most total Olympic medals? Which country has won the most gold Medals? How many Countries have 0 medals?
+ What country has had the most total athletes?
+ What sport has the most medals?

### Part 2: Prediction
+ Quantify and explain the role of a country’s GDP as it relates to the number of medals won at Olympic games.
+ Using any method you choose in Python, please extrapolate GDP through 2020 and predict the number of medals for each country at the 2018 and 2020 Olympics.
+ Discuss the assumptions you have made in the above two questions.

### Part 3: Open Ended Analysis
+ Identify patterns any additional patterns in the data with your discretion. Try to find noteworthy or unexpected patterns with any additional time.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid") 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


import pandas_profiling as pp

In [2]:
# Olympic Medal Data
OlympicMedal_DF = pd.read_csv('https://www.dropbox.com/s/511s9eaxep8z2y5/InterviewOlympicData.csv?dl=1')
# Country GDP Data per Capita
CountryGDPData_DF = pd.read_csv('https://www.dropbox.com/s/lroxbkalfxcuvmk/InterviewGDPperCapita.csv?dl=1',  encoding = "latin1")
# Mapping Table
CountryCodeMapping_DF = pd.read_csv('https://www.dropbox.com/s/0drr5fgr7e4dwqh/CountryCodeMapping.csv?dl=1')

# bad network, use local file
# OlympicMedal_DF = pd.read_csv('InterviewOlympicData.csv')
# CountryGDPData_DF = pd.read_csv('InterviewGDPperCapita.csv',  encoding = "latin1")
# CountryCodeMapping_DF = pd.read_csv('CountryCodeMapping.csv')

### View data

In [3]:
OlympicMedal_DF.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [4]:
OlympicMedal_DF[['ID', 'Name', 'Sport', 'Year', 'Season', 'Medal', 'NOC']].head()

,ID,Name,Sport,Year,Season,Medal,NOC
0,1,A Dijiang,Basketball,1992,Summer,NaN,CHN
1,2,A Lamusi,Judo,2012,Summer,NaN,CHN
2,3,Gunnar Nielsen Aaby,Football,1920,Summer,NaN,DEN
3,4,Edgar Lindenau Aabye,Tug-Of-War,1900,Summer,Gold,DEN
4,5,Christine Jacoba Aaftink,Speed Skating,1988,Winter,NaN,NED


In [5]:
CountryGDPData_DF.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,27546.899390,24631.434860,24271.940420,25324.720360,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,59.777327,59.878153,58.492874,78.782758,82.208444,101.290471,...,373.361116,445.893298,553.300289,603.537023,669.009051,638.612543,629.345250,569.577923,561.778746,585.850064
2,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3868.579014,3347.844900,3531.416878,4299.008136,4539.467689,4804.633826,4707.578098,3683.552607,3308.772828,4170.312280
3,Albania,ALB,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4370.539647,4114.136545,4094.358832,4437.178068,4247.614308,4413.081697,4578.666728,3952.830781,4131.872341,4537.862492
4,Andorra,AND,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,47785.659090,43339.379870,39736.354060,41098.766940,38391.080870,40619.711300,42294.994730,36038.267600,37231.815670,39146.548840


In [6]:
CountryCodeMapping_DF.head()

,NOC,ISO3
0,AFG,AFG
1,ANT,ATG
2,ALB,ALB
3,ALG,DZA
4,AND,AND


In [7]:
report = pp.ProfileReport(OlympicMedal_DF)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Merge data, get used data

In [8]:
# merge
OlympicMedal_DF_merge = pd.merge(OlympicMedal_DF, CountryCodeMapping_DF, how='left', on='NOC')

In [9]:
OlympicMedal_DF_merge = pd.merge(OlympicMedal_DF, CountryCodeMapping_DF, how='left', on='NOC')

olympic_df = OlympicMedal_DF_merge[['ID', 'Name', 'Sport', 'Year', 'Season', 'Medal', 'ISO3']]
olympic_df = olympic_df.rename(columns={'ISO3': 'Country Code'})
gdp_df = CountryGDPData_DF.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis=1)

In [10]:
olympic_df

,ID,Name,Sport,Year,Season,Medal,Country Code
0,1,A Dijiang,Basketball,1992,Summer,NaN,CHN
1,2,A Lamusi,Judo,2012,Summer,NaN,CHN
2,3,Gunnar Nielsen Aaby,Football,1920,Summer,NaN,DNK
3,4,Edgar Lindenau Aabye,Tug-Of-War,1900,Summer,Gold,DNK
4,5,Christine Jacoba Aaftink,Speed Skating,1988,Winter,NaN,NLD
...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,Luge,1976,Winter,NaN,POL
271112,135570,Piotr ya,Ski Jumping,2014,Winter,NaN,POL
271113,135570,Piotr ya,Ski Jumping,2014,Winter,NaN,POL
271114,135571,Tomasz Ireneusz ya,Bobsleigh,1998,Winter,NaN,POL


# Part 1: Data Description

## What do you notice about the quality of the data? Why do you think this is the case?

### No country mapping problem

Several countried don't have valid ISO3 mapping.

In [11]:
df = OlympicMedal_DF_merge[['ISO3', 'NOC']]

df_iso3_null = df[pd.isnull(df['ISO3'])]
df_noc_null = df[pd.isnull(df['NOC'])]

print("Countries taht don't have country code mapping for ISO3:\n", df_iso3_null['NOC'].unique())
df_iso3_null


Countries taht don't have country code mapping for ISO3:
 ['URS' 'UAR' 'YAR' 'SGP' 'EUN' 'FRG' 'GDR' 'TCH' 'TTO' 'VIN' 'AHO' 'YMD'
 'ANZ' 'SCG' 'IOA' 'YUG' 'MAL' 'LAO' 'ROT' 'CPV' 'CRT' 'SAA' 'RHO' 'STP'
 'MKD' 'BOH' 'WIF' 'SWZ' 'NBO' 'VNM' 'UNK' 'NFL' 'KOS']


,ISO3,NOC
192,NaN,URS
193,NaN,URS
238,NaN,UAR
302,NaN,UAR
303,NaN,UAR
...,...,...
271078,NaN,URS
271079,NaN,URS
271080,NaN,URS
271100,NaN,TCH


In [12]:
print("Countries taht don't have country code mapping for NOC:\n", df_noc_null['ISO3'].unique())
df_noc_null

Countries taht don't have country code mapping for NOC:
 []


,ISO3,NOC


### Data Sparsity

Through data sparsity check, we have the finding:
1. For Olympic Medal Data, large portion (85.33%) atheletes don't have a medal. This is not a issue.

2. For GDP per Capita data, many countries lack large portion of historical GDP data. This is important because for time very far away from now we don't have data to get reliable analysis.

In [13]:
nan_percent = pd.isnull(olympic_df).sum(axis=0)/olympic_df.shape[0]

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent, palette='rocket')
plt.xticks(rotation=45)
plt.title('Percent of missing data: Olympic Medal Data')

nan_percent.to_frame().sort_values(by=0, ascending=False).rename(columns={0: 'Missing value percent: Olympic Medal Data'}).style.format("{:.2%}")

,Missing value percent: Olympic Medal Data
Medal,85.33%
Country Code,7.98%
ID,0.00%
Name,0.00%
Sport,0.00%
Year,0.00%
Season,0.00%


In [14]:
nan_percent = pd.isnull(gdp_df.set_index('Country Code')).sum(axis=1)/gdp_df.shape[1]

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent, palette='rocket')
plt.xticks(rotation=45)
plt.title('Percent of missing data: GDP per Capita data')

nan_percent.to_frame().sort_values(by=0, ascending=False).rename(columns={0: 'Missing value percent: GDP per Capita data'}).style.format("{:.2%}")

,Missing value percent: GDP per Capita data
Country Code,
GIB,98.31%
MAF,98.31%
PRK,98.31%
TCA,98.31%
VGB,98.31%
CUW,98.31%
INX,98.31%
SXM,98.31%
CYM,94.92%


## Which country has won the most total Olympic medals? Which country has won the most gold Medals? How many Countries have 0 medals?

In [15]:
# drop no medal athelets
medal_num = olympic_df.groupby('Country Code').count()['Medal'].sort_values(ascending=False)
print(f'The country has won the most total Olympic medals: {medal_num.index[0]}: {medal_num.iloc[0]}')

The country has won the most total Olympic medals: USA: 5637


In [16]:
olympic_df_gold = olympic_df[olympic_df['Medal']=='Gold']
gold_num = olympic_df_gold.groupby('Country Code').count()['Medal'].sort_values(ascending=False)
print(f'The country has won the most gold medals: {gold_num.index[0]}: {gold_num.iloc[0]}')

The country has won the most gold medals: USA: 2638


In [17]:
medal_num = olympic_df.groupby('Country Code').count()['Medal']
no_medal_country_num = np.sum(medal_num==0)
print(f'Number of countries have 0 medals: {no_medal_country_num}')

Number of countries have 0 medals: 65


## What country has had the most total athletes?

In [18]:
athelete_num = olympic_df.groupby('Country Code')['ID'].nunique().sort_values(ascending=False)
print(f'The country has the most total athletes: {athelete_num.index[0]}: {athelete_num.iloc[0]}')

The country has the most total athletes: USA: 9653


## What sport has the most medals?

In [19]:
medal_num_by_sport = olympic_df.groupby('Sport').count()['Medal'].sort_values(ascending=False)
print(f'The Sport has the most gold medals: {medal_num_by_sport.index[0]}: {medal_num_by_sport.iloc[0]}')

The Sport has the most gold medals: Athletics: 3969


# Part 2: Prediction

## Quantify and explain the role of a country’s GDP as it relates to the number of medals won at Olympic games.

In [20]:
# fill gdp values
olympic_df['GDP'] = np.nan
gdp_df2 = gdp_df.set_index('Country Code')

country_set = set(gdp_df2.index)
year_set = set(gdp_df2.columns)

for i in range(len(olympic_df)):
    year = olympic_df['Year'].iloc[i]
    country = olympic_df['Country Code'].iloc[i]
    
    if pd.isnull(country) or pd.isnull(year):
        continue
    if str(year) in year_set and country in country_set:
        olympic_df['GDP'].iloc[i] = gdp_df2.loc[country, str(year)]


D:\programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [21]:
medal_gdp_each_point = olympic_df.groupby(['Country Code', 'Year']).agg({'GDP': 'mean', 'Medal': 'count'}).dropna()
gdp_each_point = medal_gdp_each_point['GDP']
medal_each_point = medal_gdp_each_point['Medal']
medal_gdp_each_point

GDP  Medal
Country Code Year                     
ABW          1996  16585.195530      0
             2000  20620.700630      0
             2004  22567.824920      0
             2008  27546.899390      0
AFG          1960     59.777327      0
...                         ...    ...
ZWE          2004    454.360665      3
             2008    325.678570      4
             2012    968.163875      0
             2014   1031.104614      0
             2016   1029.076649      0

[2318 rows x 2 columns]

### Correlation

There is little correlation between gdp and number of medals.

In [22]:
corr = np.corrcoef(gdp_each_point, medal_each_point)[0, 1]
print("Correlation between gdp and number of medals:\n", corr)


Correlation between gdp and number of medals:
 0.19690297263151116


### Regression

Through regression, we have:
1. t value is significant, we can reject that GDP's coefficient is zero.
2. For each value increase in GDP, the number of medals will increase 0.0003.

In [23]:
model1 = smf.ols(formula='Medal ~ GDP', data=medal_gdp_each_point).fit() 
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  Medal   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     93.41
Date:                Sat, 08 Jan 2022   Prob (F-statistic):           1.08e-21
Time:                        23:39:10   Log-Likelihood:                -10920.
No. Observations:                2318   AIC:                         2.184e+04
Df Residuals:                    2316   BIC:                         2.186e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.0210      0.645      9.339      0.0

## Using any method you choose in Python, please extrapolate GDP through 2020 and predict the number of medals for each country at the 2018 and 2020 Olympics.

From the regression, we have already got the model. Then we can give each country's GDP for each year into the model and get y_hat as our predicted number of medals.

### Extrapolate GDP
I will simply use regression model to predict GDP. $GDP=a+bt+ct^2$

In [24]:
extrapolate_years = list(range(2017, 2021))
extrapolate_years_str = [str(x) for x in extrapolate_years]
for year in extrapolate_years:
    gdp_df2[str(year)] = np.nan

for country in list(gdp_df2.index):
    data = gdp_df2.loc[country,:].dropna().to_frame()
    if not len(data):
        continue
    data['t'] = data.index
    data['t'] = data['t'].apply(int)
    data['t2'] = data['t']**2

    data = data.rename(columns={country: 'GDP'})
    reg = LinearRegression().fit(data[['t','t2']], data['GDP'])
    t = [[x, x**2] for x in extrapolate_years]
    y_hat = reg.predict(t)
    gdp_df2.loc[country, extrapolate_years_str] = y_hat
    
gdp_df2

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Code,,,,,,,,,,,,,,,,,,,,,
ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25324.720360,NaN,NaN,NaN,NaN,NaN,28377.615803,28686.425455,28976.108223,29246.664108
AFG,59.777327,59.878153,58.492874,78.782758,82.208444,101.290471,137.899362,161.322000,129.506654,129.798541,...,603.537023,669.009051,638.612543,629.345250,569.577923,561.778746,573.232170,590.734567,608.574118,626.750822
AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4299.008136,4539.467689,4804.633826,4707.578098,3683.552607,3308.772828,5380.508252,5763.501003,6160.424927,6571.280025
ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4437.178068,4247.614308,4413.081697,4578.666728,3952.830781,4131.872341,5565.488476,5890.022736,6224.517907,6568.973989
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41098.766940,38391.080870,40619.711300,42294.994730,36038.267600,37231.815670,48515.221950,50022.653054,51552.889992,53105.932766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3733.491539,3600.815825,3877.758936,4054.721339,3574.174030,3697.548026,3886.402737,3886.898286,3868.729714,3831.897021
YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1349.420222,1421.170984,1580.181689,1647.033586,1285.563019,660.280885,1549.392118,1622.265942,1697.021058,1773.657465
ZAF,433.941194,444.896268,461.798232,498.601747,534.374176,568.348265,603.069811,654.626304,688.830622,755.181916,...,7976.466077,7478.227665,6822.524760,6433.944544,5746.681127,5280.017633,6959.462908,7138.910533,7320.565981,7504.429253


### Use regression model between GDP and MedalNumbers to predict

In [25]:
result = []
for country in list(gdp_df2.index):
    for year in [2018, 2020]:
        gdp = gdp_df2.loc[country, str(year)]
        result.append([country, year, gdp])
result_df = pd.DataFrame(result, columns=['Country Code', 'Year', 'GDP'])
result_df['MedalsNum'] = model1.predict(result_df[['GDP']])
result_df

,Country Code,Year,GDP,MedalsNum
0,ABW,2018,28686.425455,14.326938
1,ABW,2020,29246.664108,14.489151
2,AFG,2018,590.734567,6.192041
3,AFG,2020,626.750822,6.202469
4,AGO,2018,5763.501003,7.689777
...,...,...,...,...
523,ZAF,2020,7504.429253,8.193849
524,ZMB,2018,1642.126817,6.496463
525,ZMB,2020,1774.381595,6.534757
526,ZWE,2018,666.217185,6.213896


## Discuss the assumptions you have made in the above two questions.

For the GDP-MedalsNum regression model, several assumption are made:
1. The GDP in different countries are comparable. GDP and residuals are independent. This also means country factor is not an ommitted variable in the model. No other ommitted variables.
2. Relation between GDP and number of medals are linear.
3. No residual autocorrelation.

For the extroplate regression model, several assumptions are made:
1. Relation between GDP and time and time^2 are linear.
2. Independent variables and residuals are independent. No ommitted variables.


# Part 3: Open Ended Analysis

## Identify patterns any additional patterns in the data with your discretion. Try to find noteworthy or unexpected patterns with any additional time.

### GDP-MedalNumber relationship differs greatly among countries

For example:
- USA: slightly positive coefficients.
- DEU: slightly negative coefficients.
- CHN: large positive coefficients.

This means we should introduce country factor in our model to predict number of medals.

In [26]:
model_USA = smf.ols(formula='Medal ~ GDP', data=medal_gdp_each_point.loc['USA']).fit() 
print(model_USA.summary())

                            OLS Regression Results                            
Dep. Variable:                  Medal   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.052
Method:                 Least Squares   F-statistic:                  0.008116
Date:                Sat, 08 Jan 2022   Prob (F-statistic):              0.929
Time:                        23:39:11   Log-Likelihood:                -126.03
No. Observations:                  21   AIC:                             256.1
Df Residuals:                      19   BIC:                             258.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    170.8649     43.194      3.956      0.0

In [27]:
model_DEU = smf.ols(formula='Medal ~ GDP', data=medal_gdp_each_point.loc['DEU']).fit() 
print(model_DEU.summary())

                            OLS Regression Results                            
Dep. Variable:                  Medal   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                 -0.056
Method:                 Least Squares   F-statistic:                    0.3632
Date:                Sat, 08 Jan 2022   Prob (F-statistic):              0.559
Time:                        23:39:11   Log-Likelihood:                -70.730
No. Observations:                  13   AIC:                             145.5
Df Residuals:                      11   BIC:                             146.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    139.8157     72.145      1.938      0.0

D:\programs\Anaconda\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [28]:
model_CHN = smf.ols(formula='Medal ~ GDP', data=medal_gdp_each_point.loc['CHN']).fit() 
print(model_CHN.summary())

                            OLS Regression Results                            
Dep. Variable:                  Medal   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.8436
Date:                Sat, 08 Jan 2022   Prob (F-statistic):              0.374
Time:                        23:39:11   Log-Likelihood:                -85.584
No. Observations:                  16   AIC:                             175.2
Df Residuals:                      14   BIC:                             176.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     50.3806     18.441      2.732      0.0

D:\programs\Anaconda\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### GDP-MedalNumber relationship differs greatly among periods of time

The coefficient is larger for data before 2020. So we should introduce time related variables in the model.

In [29]:
medal_gdp = medal_gdp_each_point.reset_index(drop=False)
medal_gdp_before_2000 = medal_gdp[medal_gdp['Year']<2000]
medal_gdp_after_2000 = medal_gdp[medal_gdp['Year']>=2000]

In [30]:
model_b2000 = smf.ols(formula='Medal ~ GDP', data=medal_gdp_before_2000).fit() 
print(model_b2000.summary())

                            OLS Regression Results                            
Dep. Variable:                  Medal   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     47.90
Date:                Sat, 08 Jan 2022   Prob (F-statistic):           7.62e-12
Time:                        23:39:12   Log-Likelihood:                -5126.5
No. Observations:                1099   AIC:                         1.026e+04
Df Residuals:                    1097   BIC:                         1.027e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.7594      0.879      6.554      0.0

In [31]:
model_a2000 = smf.ols(formula='Medal ~ GDP', data=medal_gdp_after_2000).fit() 
print(model_a2000.summary())

                            OLS Regression Results                            
Dep. Variable:                  Medal   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     54.99
Date:                Sat, 08 Jan 2022   Prob (F-statistic):           2.26e-13
Time:                        23:39:12   Log-Likelihood:                -5785.0
No. Observations:                1219   AIC:                         1.157e+04
Df Residuals:                    1217   BIC:                         1.158e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.6393      0.959      5.880      0.0